# Veeper: Create, Run, and Collate

This is a generalized code that can bring in any spectra, save it to its own components folder, and save the corresponding fits

Install: https://github.com/mattcwilde/veeper

This notebook is supposed to be run after asborption features have been identified using pyigm_igm guesses

Install: https://github.com/pyigm/pyigm

Here are some notes for the future

#### Running this code in Step 3 will produce a FitInspection.pdf file with a preview of your voigt fits: 
- Red line = total 
- dashed line = particular component 
- the order is the redshift 

#### If the file needs to be changed (editing.ipynb): 

In [1]:
##--- Step one 
from joebvp.utils import pyigm_to_veeper
from joebvp.VPmeasure import batch_fit
from joebvp import VPmeasure
import glob
import os
import astropy.units as u

read_sets: Using set file -- 
  /Users/samgarza/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
joebvp.VPmeasure: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, usin

#### Paths for each of my spectra: 
- ##### NGC 3489 / J1101: 
    - json_path =  '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1101+1429/SamIDs_jess_Nov29_2022.json' 
    - norm = 'J1101+1429_nbin3_jwnorm.fits'

- ##### NGC 4026 / J1159: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/SamIDs_meth3_Dec1_2022_broad.json'
    - norm = 'J1159+5106_nbin3_jwnorm_meth3.fits'
    
- ##### NGC 4258 / J1222: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1222+4612/SamIDs_Jan5_2022.json'
    - norm = 'J1222+4612_nbin3_jwnorm.fits'
    
- ##### NGC 4736 / J1249: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/SamIDs_Jan6_2022.json'
    - norm = 'J1249+4122_nbin3_jwnorm.fits'
    
- ##### NGC 3414 / 1051: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1051+2805/SamIDs_Nov29_2022.json'
    - norm = 'J1051+2805_nbin3_jwnorm.fits'
    
- ##### NGC 3627 / J1123: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Dec1_2022_mwfix.json'
    - norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'

- ##### NGC 1097 / J0246: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/SamIDs_Nov28_2022.json'
    - norm = 'J0246-3007_nbin3_allvisits_jwnorm.fits'

- ##### NGC 4564 / J1235: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/SamIDs_Nov28_2022.json'
    - norm = 'J1235+1123_twovisits_m3_jwnorm.fits'

In [14]:
#Step 2: Imput the specifications 
    # basically takes the gueses and the norm to start the process
    #dv_expand is how much you can expand the redlines to be of some problem 

#These will change for each galaxy/line of sight  
#path to specific .json file
json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/SamIDs_Jan6_2022_ii.json'

# normalization file that you want to use
norm = 'J1249+4122_nbin3_jwnorm.fits'
norm_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/J1249+4122_nbin3_jwnorm.fits' #path to normalization file 

pyigm_to_veeper(json_path, norm, dv_expand=50*u.km/u.s)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [15]:
input_files = glob.glob('component_groups/*.txt')

#This was 10 in the other one but I want to make this 20 since it skipped something that was 15 before 
max_len = 20 #skip over files where there are more than this many lines to fit 
redo = False

to_do = []
for input_file in input_files:
    if 'rchi2' in input_file:
        continue
    if not redo:
        #already done?
        if os.path.exists(input_file.replace('.txt', '.VP')): 
            print('Skipping: Already done {}'.format(input_file))
            continue
    
    #too long?
    with open(input_file, 'r') as f:
        length = len(f.readlines())
        if length > max_len:
            print('Skipping: Too long ({} rows) {}'.format(length, input_file))
            continue
            
    to_do.append(input_file)

In [16]:
###--- Step 3: Run the fit
VPmeasure.batch_fit(norm_path, filelist=to_do, filepath='./component_groups/')

`xtol` termination condition is satisfied.
Function evaluations 21, initial cost 5.7781e+01, final cost 1.9074e+01, first-order optimality 1.02e-02.

Fit results: 

1670.79	 -0.000180	 15.043	 20.016	 8.8132
 	  	  	 1.701	 7.929	 1.052 


Reduced chi-squared: 0.669274
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_8.txt
Line parameters written to:
component_groups/input_group_8.VP
Wrote spectrum to component_groups/input_group_8_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_8_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 12, initial cost 2.5158e+02, final cost 2.1320e+02, first-order optimality 1.21e-02.

Fit results: 

1393.76	 -0.000170	 13.597	 63.300	 31.196
 	  	  	 0.027	 4.842	 3.445 

1402.77	 -0.000170	 13.597	 63.300	 31.196
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 5.537690
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_0.txt
Line parameters written to:
component_groups/input_group_0.VP
Wrote spectrum to component_groups/input_group_0_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_0_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 9, initial cost 2.4530e+02, final cost 6.3866e+01, first-order optimality 3.64e-03.

Fit results: 

1031.93	 0.368790	 14.366	 24.256	 -19.474
 	  	  	 0.02	 1.366	 0.949 

1037.62	 0.368790	 14.366	 24.256	 -19.474
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 1.680691
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_1.txt
Line parameters written to:
component_groups/input_group_1.VP
Wrote spectrum to component_groups/input_group_1_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_1_inspect.fits
`xtol` termination condition is satisfied.
Function evaluations 19, initial cost 4.2967e+01, final cost 2.9677e+01, first-order optimality 3.96e-03.

Fit results: 

1526.71	 -0.000140	 15.289	 29.190	 6.7465
 	  	  	 0.442	 5.719	 1.669 


Reduced chi-squared: 1.348975
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_3.txt
Line parameters written to:
component_groups/input_group_3.VP
Wrote spectrum to component_groups/input_group_3_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_3_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 6, initial cost 4.8461e+01, final cost 3.9806e+01, first-order optimality 1.98e-03.

Fit results: 

1215.67	 0.251390	 13.852	 90.000	 -27.456
 	  	  	 0.041	 10.046	 7.273 


Reduced chi-squared: 1.075836
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_2.txt
Line parameters written to:
component_groups/input_group_2.VP
Wrote spectrum to component_groups/input_group_2_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


...

Wrote spectrum to component_groups/input_group_5_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 6, initial cost 1.7268e+01, final cost 1.5252e+01, first-order optimality 1.32e-03.

Fit results: 

1215.67	 0.265870	 13.814	 39.519	 5.6043
 	  	  	 0.041	 4.719	 3.315 


Reduced chi-squared: 0.726281
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_4.txt
Line parameters written to:
component_groups/input_group_4.VP
Wrote spectrum to component_groups/input_group_4_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_4_inspect.fits

VPmeasure: 8/9 fits converged, 0/9 failed (see log for details).

VPmeasure: concatenating individual outputs and creating figures for inspection.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(),

Wrote spectrum to FitInspection.fits
VPmeasure: Done.


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


In [17]:
#make the directory to hold creates files for each qso/LOS 

parent_path = '/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/' #where I want the created files to be put in general 
qso_name = 'J1249_ii/' #need to change this to match each QSO LOS name; where I want the created files to be put in specifically 

#set directory name
directory = qso_name
# Path
path = os.path.join(parent_path, directory)
#Make it happen
os.mkdir(path)

In [18]:
#Step 4: Collate
import pandas as pd

vp_file_list = glob.glob('component_groups/*.VP')
vp_file_list

['component_groups/input_group_2.VP',
 'component_groups/input_group_6.VP',
 'component_groups/input_group_3.VP',
 'component_groups/input_group_8.VP',
 'component_groups/input_group_4.VP',
 'component_groups/input_group_0.VP',
 'component_groups/input_group_1.VP',
 'component_groups/input_group_5.VP']

In [19]:
vp_dfs = []

for vp_file in vp_file_list:
    vp_df = pd.read_csv(vp_file, sep='|')
    vp_dfs.append(vp_df)
    
all_vp_df = pd.concat(vp_dfs, ignore_index=True)
all_vp_df

,specfile,restwave,zsys,col,sigcol,bval,sigbval,vel,sigvel,nflag,...,vlim1,vlim2,wobs1,wobs2,pix1,pix2,z_comp,trans,rely,comment
0,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1215.6700,0.25139,13.852,0.041,90.000,10.046,-27.456,7.273,2,...,-207.7595,207.7595,1520.432134,1522.117080,3746,3792,0.25127,H I,c,--
1,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1548.2040,-0.00008,14.189,0.032,41.057,3.673,-4.499,2.573,2,...,-63.7031,63.7031,1547.752156,1548.410113,4490,4508,-0.00009,C IV,a,--
2,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1550.7810,-0.00008,14.189,0.000,41.057,0.000,-4.499,0.000,2,...,-63.7031,63.7031,1550.328404,1550.987457,4561,4579,-0.00009,C IV,a,--
3,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1548.2040,0.00052,13.863,0.052,36.278,6.521,11.632,4.127,3,...,-52.7826,52.7826,1548.729687,1549.274852,4517,4532,0.00055,C IV,b,--
4,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1550.7810,0.00052,13.863,0.000,36.278,0.000,11.632,0.000,3,...,-52.7826,52.7826,1551.307562,1551.853635,4587,4602,0.00055,C IV,b,--
5,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1548.2040,0.00074,13.635,0.134,9.357,4.982,17.237,2.532,4,...,-23.6612,23.6612,1549.227866,1549.472250,4531,4537,0.00080,C IV,c,--
6,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1550.7810,0.00074,13.635,0.000,9.357,0.000,17.237,0.000,4,...,-23.6612,23.6612,1551.806570,1552.051361,4601,4608,0.00080,C IV,c,--
7,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1548.2040,0.00114,13.963,0.044,37.160,5.363,-26.581,3.516,5,...,-86.4542,86.4542,1549.528643,1550.421586,4539,4563,0.00106,C IV,b,--
8,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1550.7810,0.00114,13.963,0.000,37.160,0.000,-26.581,0.000,5,...,-86.4542,86.4542,1552.107848,1553.002277,4609,4633,0.00106,C IV,b,--
9,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249...,1526.7070,-0.00014,15.289,0.442,29.190,5.719,6.747,1.669,2,...,-97.3189,97.3189,1525.997614,1526.988816,3897,3924,-0.00012,SiII,a,--


In [20]:
#Save collate informtion to csv file 
all_vp_df.to_csv(path + 'all_vp.csv')

In [21]:
import shutil

#Move over the component_groups
shutil.move(parent_path + 'component_groups/', path)

'/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/J1249_ii/component_groups'

In [22]:
#list of created files that need to be moved over to QSO LOS specific directory 
files_to_move = ['_VP_log.dat', 'all_VP.txt', 'compiledVPoutputs.dat', 'FitInspection.fits', 'FitInspection.pdf']

# iterate files
for file in files_to_move:
    # construct full file path
    source = parent_path + file
    destination = path + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)

Moved: _VP_log.dat
Moved: all_VP.txt
Moved: compiledVPoutputs.dat
Moved: FitInspection.fits
Moved: FitInspection.pdf
